In [1]:
import cv2

import numpy as np

In [3]:
cv2.namedWindow("preview")
vc = cv2.VideoCapture(0)


Ball_MIN = np.array([35, 50, 6], np.uint8)
Ball_MAX = np.array([50, 255, 255],np.uint8)

if vc.isOpened(): # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

track = []
start = False
while rval:
    
    rval, frame = vc.read()
    frame = cv2.resize(frame, (0,0), fx = 0.75, fy = 0.75)
    key = cv2.waitKey(10)
    if key == 27: # exit on ESC
        break
    frame = cv2.GaussianBlur(frame, (11,11), 2)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    threshold = cv2.inRange(hsv,Ball_MIN , Ball_MAX)
    threshold = cv2.erode(threshold, None, iterations=2)
    threshold = cv2.dilate(threshold, None, iterations=2)
    
    cnt, heir = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    ball = []
    
    if len(cnt) > 0 :
        #ball = max(cnt, key=cv2.contourArea)
        for c in cnt:
            approx = cv2.approxPolyDP(c,0.01*cv2.arcLength(c,True),True)
            area = cv2.contourArea(c)
            if ((len(approx) > 8) & (area > 250)):
                ball = c
                val = cv2.minEnclosingCircle(c)
                track.append(list(val[0]))
                #print(track)
                break
        if len(ball) > 0:
            pass
            #cv2.drawContours(frame, ball, -1, (0, 255, 0), 2)
    else:
        if len(track) > 0:
            track.pop(0)
    
    while len(track) > 50:
        track.pop(0) 
        start = True
    #print(type(track))
    
    if len(track) > 0 and start:
        track.pop(0)
    else:
        start = False
    
    
    
    for i in range(2, len(track), 3):
        start1 = np.array(list(track[i-2]), dtype=int)
        end1 = np.array(list(track[i]), dtype=int)
        
        cv2.line(frame, start1, end1, (255,0,0), 2)
    
    cv2.imshow("preview", threshold)
    cv2.imshow("preview1", frame)

vc.release()
cv2.destroyAllWindows()

In [ ]:
# img = cv2.imread("test.jpg")
img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

In [11]:
img[300,250]

array([102,  83,  77], dtype=uint8)